In [11]:
import json
import tqdm
import hashlib
from typing import (
    Union,
    Sequence
)
import math
import random
import sympy as sp


In [20]:

# function hub
def read_json_file(file_path): 
    ''' opens a json file, return its content save in a dictionary
    @input: file path to the json file
    @output: a dictionary of json file content
    '''
    file = open(file_path, 'r')
    values = json.load(file)
    file.close()
    return values

def hash_elems(*a):
    
    
    h = hashlib.sha256()
    h.update("|".encode("utf-8"))
    
    for x in a:

        if not x:
            # This case captures empty lists and None, nicely guaranteeing that we don't
            # need to do a recursive call if the list is empty. So we need a string to
            # feed in for both of these cases. "None" would be a Python-specific thing,
            # so we'll go with the more JSON-ish "null".
            hash_me = "null"

        elif isinstance(x, str):
            # strings are iterable, so it's important to handle them before the following check
            hash_me = x
        elif isinstance(x, Sequence):
             # The simplest way to deal with lists, tuples, and such are to crunch them recursively.
            hash_me = str(hash_elems(*x))           
        else:
            hash_me = str(x)
        h.update((hash_me + "|").encode("utf-8"))

    # Note: the returned value will range from [1,Q), because zeros are bad
    # for some of the nonces. (g^0 == 1, which would be an unhelpful thing
    # to multiply something with, if you were trying to encrypt it.)

    # Also, we don't need the checked version of int_to_q, because the
    # modulo operation here guarantees that we're in bounds.
    # return int_to_q_unchecked(
    #     1 + (int.from_bytes(h.digest(), byteorder="big") % Q_MINUS_ONE)
    # )

    return 1 + (int.from_bytes(h.digest(), byteorder="big") % (q - 1))

def mod_p(n):
    if isinstance(n, str):
        n = int(n) 
    return n % p

def is_valid_of_Z_boundary(n, boundary):
    is_valid = True
    if isinstance(n, str):
        n = int(n) 
    if n < 0 or n >= boundary:
        is_valid = False
    return is_valid   
    
def is_valid_of_Zr_boundary(n, boundary, alpha):
    is_valid = True
    if isinstance(n, str):
        n = int(n)    
    if n < 0 or n >= boundary:
        is_valid = False
    
    n_power = pow(alpha, q, p)#?
    if n_power != 1:
        is_valid = False
    return is_valid


def equals(a, b):
    ''''compares two values
    @input: two integers a, b 
    @output: True if a, b have same values, False otherwise
    '''
    return (a == b)

def get_length(num):
    ''' get the length of an integer using log
    @input: a positive integer
    @output: its length 
    '''
    len_of_num = int(math.log(num, 10) + 1)
    return len_of_num

# the following code is adapted from GeeksforGeeks
def power_mod(x, y, p): 
    ''' do modular exponentiation. 
    @input: x - base, 
            y - exponent,
            m - modulus
    @output: integer result of x^y % m
    '''
    # Initialize result 
    res = 1
      
    # Update x if it is more than or equal to p 
    if(x >= p):
        x = x % p

    # positive exponential
    while (y > 0): 
          
        # If y is odd, multiply x with result 
        if (y & 1): 
            res = (res * x) % p
  
        # y must be even now 
        y = y >> 1; # y = y/2 
        x = (x * x) % p
    return res
    
def __miller_test(d, num):
    ''' find a odd number of d such that num - 1 = d * 2^r
    @input: d - a odd number that num - 1 = d * 2^r for r >= 1
            num - the number needs to be check against
    @output: True if num is prime, False if it's a composite
    '''
    # Pick a random number in [2..n-2] 
    # Corner cases make sure that n > 4 
    a = 2 + random.randint(1, num - 4)

    # Compute a^d % n 
    x = power_mod(a, d, num)
  
    if (x == 1 or x == num - 1): 
        return True
  
    # Keep squaring x while one of the following doesn't happen 
    # (i) d does not reach n-1 
    # (ii) (x^2) % n is not 1 
    # (iii) (x^2) % n is not n-1 
    while (d != num - 1): 
        x = (x * x) % num
        d *= 2
  
        if (x == 1): 
            return False; 
        if (x == num - 1): 
            return True
  
    # Return composite 
    return False

def is_prime(num, k): 
    ''' implements Miller-Rabin algorithm to test the primality of a number
    @input: num - a positive integer
            k - the number of iterations, impacting accuracy
    @output: True if it's a prime, False otherwise 
    '''
    # Corner cases 
    if (num <= 1 or num == 4): 
        return False
    if (num <= 3): 
        return True
  
    # Find r such that n = 2^d * r + 1 for some r >= 1 
    d = num - 1
    while (d % 2 == 0): 
        d //= 2
  
    # Iterate given number of 'k' times 
    for i in range(k): 
        if (__miller_test(d, num) == False): 
            return False
  
    return True

def is_divisor(a, b): 
    '''check if a is a divisor of b
    @input: a, b - positive integers 
    @output: True if a is a divisor of b, False otherwise'''
    return (a % b == 0)

def create_hash_parameter_list(i): 
    '''create a list of parameters needed for hash computation
    @input: i - the i-th trustee/guardian
            base-hash - given base hash code
    @output: a list of all the parameters needed, including base hash Q, Ki,j, hi,j
    '''
    # invalid index input
    if (i < 0 or i > 4): 
        raise ValueError('i should be within range of 0 - 4.')

    else:
        # declare variables
        coefficient_file_path = ''
        param_list = []

        #append base hash to list
        #param_list.append(base_hash)

        # get file name dynamically
        coefficients_file_path = ('results/coefficients/coefficient_validation_set_hamilton-county-canvass-board-member-' + 
                                  str(i) + '.json')
        
        
        # read file 
        coefficients = read_json_file(coefficients_file_path)

        # get all the commitment values Ki,j and append to list 
        commitments = coefficients['coefficientCommitments']
        for commitment in commitments: 
            param_list.append(commitment)
        
        # get all the hi,j values and append to list
        proofs = coefficients['coefficientProofs']
        for item in proofs:
            h_ij = item['h']
            param_list.append(h_ij)

        return param_list

    
def get_field_ij(i, j, field): 
    ''' getter methods to get ui,j, hi,j, ki,j
    @input: i - i th member, j - j th coefficient, field - u/h/k
    @output: a certain field of the i-th member, stored in its j-th item
    '''
    # exception
    if (i < 0 or i >= 5 or j < 0 or j >= 3):
        raise ValueError('i should be in range (0,{}), j should be in range (0, {})'
                         .format(num_of_guardian, threshold))

    # get file name dynamically according to i
    coefficients_file_path = ('results/coefficients/coefficient_validation_set_'+
    'hamilton-county-canvass-board-member-' + str(i) + '.json')
    
    coefficients = read_json_file(coefficients_file_path)
    proofs = coefficients['coefficientProofs']

    # get specific value according to j and the given field
    return int(proofs[j][field])

### Green box 1.1 Baseline parameters - overview 

In [ ]:
# TODO: needs to update paths to new results when the constants.json is given with non-null 


In [7]:

# file path configuration
constants_file_path = 'results/constants.json'
context_file_path = 'results/context.json'
ballot_file_path = 'results/encrypted_ballots/ballot_ballot-af0a2a64-c786-11ea-a311-acde48001122.json'


# basic parameters
g = 0
p = 0
q = 0
r = 0
gInv = 0
K = 0

In [8]:
constants = read_json_file(constants_file_path)
#print(constants)

g = constants['g']
p = constants['p']
q = constants['q']
q_test = pow(2, 256) - 189

r = constants['r']
gInv = constants['gInv']

#print('g:'+str(g))
#print('p:'+str(p))
#print('q:'+str(q))

context = read_json_file(context_file_path)
#print(context)

K = int(context['elgamalPublicKey'])
QBar = context['cryptoExtendedBaseHash']

print(K)


8902228262352507482376024745531485971443340210464286875179634320227592743274706098381064532883160461211626951858008519076836811880985097802420111949675931804135492341936214204781882140780957252442215380957624854841507680832366704118530338114603192117215467177814401302903243141053544520818713824008399256926707721375104566953365657924590153186853581901380188498253951405612841921342606069743021006123304700009120712081439516853013896197939798539033738691446135507942972748654043241628612519782688488132553511658831880620895588870861811987109148280056555228733158308943437556966475596801135473289552751666989626057427535108529692297275166884973597872260136554561439983154503891840348240685210820490824716553029531446855846702152825665048487339721674399980978668881945855597259493113647141520484186616308305661333443119826505716914118192019463320022197012998539863235037320622230878221413775934537552183408339688696853598918360032997804775400094302732993271962282845522421257766586314548117938780349454

### Green box 1.2 Baseline parameters - check against each election

In [21]:

error = False 

# use Miller-Rabin algorithm to check the primality of p and q
# set iteration to run 50 times by default
DEFAULT_K = 50
if not is_prime(p, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(p, DEFAULT_K), var = "p"))
if not is_prime(q, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(q, DEFAULT_K), var = "q"))

# check equation p - 1 = qr
if not equals(p - 1, q * r): 
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = equals(p - 1, q * r), equation = "p - 1 = q * r"))

# check q is not a divisor of r, and 1 < g < p
if is_divisor(q, r):
    error = True
    print('''It\'s verfied as {res} that {var1} is not a divisor of {var2}.'''
      .format(res = not(is_divisor(q, r)), var1 = "q", var2 = "r"))

# check 1 < g < p
is_within_range = ((g > 1) and (g < p))
if not is_within_range: 
    error = True
    print('''It\'s verfied as {res} that {cond}.'''
      .format(res = is_within_range, cond = "1 < g < p"))

# check g^q mod p = 1 
result_actual = power_mod(g, q, p)
result_expected = 1 
if not equals(result_actual, result_expected):
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = str(equals(result_actual, result_expected)), 
              equation = 'g^q mod p = 1'))
if not error:
    print('success')


success


### Green box 2.1 Key Generation, confirm hash computation
calculate $c_i = H(Q,K_{i,0},K_{i,1},K_{i,2},...,K_{i, k-1},h_{i,0},h_{i,1},h_{i,2},...,h_{i,k-1})$ mod q

related file: context.json

coefficients files


In [14]:

# store variables of number of trustees/guardians, threshold, 
# base hash and extended hash
num_of_guardian = context['numberOfGuardians']
threshold = context['quorum']
base_hash = context['cryptoBaseHash']
extended_hash = context['cryptoExtendedBaseHash']


In [15]:

# save all members' hash results in a list
hash_res_list = []

for i in range(0, 5): 
    param_list = create_hash_parameter_list(i)
    #print(param_list)
    hash_res = hash_elems(param_list)
    print('Guardian {i} hash: {hashcode}'.format(i = i, hashcode = hash_res))
    hash_res_list.append(hash_res)

# hash results mod q to generate ci 
ci_list = []

for i in range(0, 5):
    ci_i = hash_res_list[i] % q
    print('Guardian {i} ci : {ci}'.format(i = i, ci = ci_i))
    ci_list.append(ci_i)


Guardian 0 hash: 32146972260342168407143829831061465771988147761972258078934689775459706799457
Guardian 1 hash: 91164261318783244032670443630116779124518579471030119038698970673062460240104
Guardian 2 hash: 71258822633725847384712337525875060139852771622527542393984861403890335986253
Guardian 3 hash: 87932471679333890568144546283794883360334920171287100279785410592462406156146
Guardian 4 hash: 63879865643452282161201438840606443606606814865571043296743742666395038057840
Guardian 0 ci : 32146972260342168407143829831061465771988147761972258078934689775459706799457
Guardian 1 ci : 91164261318783244032670443630116779124518579471030119038698970673062460240104
Guardian 2 ci : 71258822633725847384712337525875060139852771622527542393984861403890335986253
Guardian 3 ci : 87932471679333890568144546283794883360334920171287100279785410592462406156146
Guardian 4 ci : 63879865643452282161201438840606443606606814865571043296743742666395038057840


### Green box 2.2 - check: $g^{u_{ij}} mod p = h_{i,j}K^{c_i}_{i,j} mod p$

In [16]:
# uses double for-loop to check, 0 <= i < 5, 0 <= j < 3
for i in range(0, num_of_guardian): 
    for j in range(0, threshold): 
        u_ij = get_field_ij(i, j, 'u')
        h_ij = get_field_ij(i, j, 'h')
        k_ij = get_field_ij(i, j, 'k')
        left = pow(g, u_ij, p)
        right = (h_ij % p * pow(k_ij, ci_list[i], p)) % p
        #print('left {l}, \nright {r} '.format(l = left, r = right))
        print('It\'s verified as {res} that {equation} when {i_name} = {i_val}, {j_name} = {j_val}'
        .format(res = equals(left, right), 
                equation = '𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝',
                i_name = 'i',
                i_val = str(i),
                j_name = 'j',
                j_val = str(j)))
                

It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 0, j = 0
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 0, j = 1
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 0, j = 2
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 1, j = 0
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 1, j = 1
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 1, j = 2
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 2, j = 0
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 2, j = 1
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 2, j = 2
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 3, j = 0
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 3, j = 1
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 3, j = 2
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 4, j = 0
It's verified as False that 𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝 when i = 4, j = 1
It's verified as Fal

In [17]:
# one sample data
ballot = read_json_file(ballot_file_path)

error = False

contests = ballot['contests']
for contest in contests:
    
    ballotSelections = contest['ballotSelections']
    for ballotSelection in ballotSelections:


        objectId = ballotSelection['objectId']

        alpha = int(ballotSelection['message']['alpha'])
        beta = int(ballotSelection['message']['beta'])

        a0 = int(ballotSelection['proof']['a0'])
        a1 = int(ballotSelection['proof']['a1'])
        b0 = int(ballotSelection['proof']['b0'])
        b1 = int(ballotSelection['proof']['b1'])
        c0 = int(ballotSelection['proof']['c0'])
        c1 = int(ballotSelection['proof']['c1'])
        v0 = int(ballotSelection['proof']['v0'])
        v1 = int(ballotSelection['proof']['v1'])

        alphaPow = pow(alpha, q, p)
        betaPow = pow(beta, q, p)
        a0Pow = pow(a0, q, p)
        b0Pow = pow(b0, q, p)
        a1Pow = pow(a1, q, p)
        b1Pow = pow(b1, q, p)

        #print("a0:"+str(a0))
        #print("alpha_1:"+str(alpha_1))
        # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.
        
        
        if (not is_valid_of_Zrp(alpha) or not is_valid_of_Zrp(beta) or not is_valid_of_Zrp(a0) 
            or not is_valid_of_Zrp(a1) or not is_valid_of_Zrp(b0) or not is_valid_of_Zrp(b1)):
            print ("alpha, beta, a0, a1, b0, b1, one of them is out of range.")
            error = True  

        # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
        # tempString = str(QBar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
        # tempList = (QBar, (alpha, beta), (a0, b0), (a1, b1))
        c = hash_elems(alpha, beta, a0, b0, a1, b1)
        #print(c)
        
        #print((c0 + c1) % q)
        
        if c != (c0 + c1) % q:
            print("c != (c0 + c1) % q.")
            error = True

        # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
        if not is_valid_of_Zq(c0) or not is_valid_of_Zq(c1) or not is_valid_of_Zq(v0) or not is_valid_of_Zq(v1):
            print("c0, c1, v0, v1, one of them is out of range.")
            error = True               

        # 1st equation: gv0=a0alphac0 mod p
        g_v0_ls = pow(g, v0, p) % p
        g_v0_rs = (a0 * pow(alpha, c0, p)) % p
    
        # 2nd equation: gv1=a1alphac1 mod p
        g_v1_ls = pow(g, v1, p) % p
        g_v1_rs = (a1 * pow(alpha, c1, p)) % p

        # 3rd equation: Kv0=b0betac0 mod p
        K_v0_ls = pow(K, v0, p) % p
        K_v0_rs = (b0 * pow(beta, c0, p)) % p
    
        # 4th equation: gc1Kv1=b1betac1 mod p
        fourth_ls = (pow(g, c1, p) * pow(K, v1, p) % p)
        fourth_rs = (b1 * pow(beta, c1, p)) % p

        # checking
        if g_v0_ls != g_v0_rs:
            print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
            error = True
        if g_v1_ls != g_v1_rs:
            print("g_v1 != a_1*alpha^c_1")
            error = True
        if K_v0_ls != K_v0_rs:
            print("K_v0 != b_0*alpha^c_0")
            error = True
        if fourth_ls != fourth_rs:
            print("g_c1 * K_v1 != (b1 * beta^c1) % p")
            error = True

                                               
if not error:
    print("success")   



success


In [22]:

# Green box 4
error = False

contests = ballot['contests']
for contest in contests:
    
    # total parameters
    big_a = int(contest['proof']['a'])
    big_b = int(contest['proof']['b'])
    big_c = int(contest['proof']['c'])
    big_v = int(contest['proof']['v'])
    big_constant = int(contest['proof']['constant'])
    big_alpha = 1
    big_beta = 1
       
    if not is_valid_of_Zq(big_v) or not is_valid_of_Zrp(big_a) or not is_valid_of_Zrp(big_b):
        error = True
        print("big_a, big_b, big_v, one of them is out of range.")
  
    ballotSelections = contest['ballotSelections']
    for ballotSelection in ballotSelections:
        
        alpha = int(ballotSelection['message']['alpha'])
        beta = int(ballotSelection['message']['beta'])  
        big_alpha = big_alpha * alpha % p
        big_beta = big_beta * beta % p
    
    temp_c = hash_elems(big_alpha, big_beta, big_a, big_b)
    print(temp_c)
    print(big_c)
    
    
    # g power v mod p = a * A power c mod p
    g_v_ls = pow(g, big_v, p)
    a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))
    
    #print(g_v_ls)
    #print(a_ac_rs)
    if g_v_ls != a_ac_rs:
        error = True
        #print("g_v_ls is not equal a_ac_rs.")
    
    # g power Lc mod p * K power v mod p = b * B power c mod p
    g_l_k_ls = mod_p((mod_p(g)) * (pow(K, big_v, p)))
    b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))
    
    
    # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
    # mult_p(b, pow_p(beta, c))
    #print(g_l_k_ls)
    #print(b_bc_rs)
    if g_l_k_ls != b_bc_rs:
        error = True
        #print("g_l_k_ls is not equal b_bc_rs.")        
    
if not error:
    print("success")        
    


NameError: name 'ballot' is not defined

In [19]:
context = read_json_file("../data_08042020/context.json")
QBar = context['crypto_extended_base_hash']

In [14]:
# green box 6 

# read in tally file
tally_file_path = "new_results/tally.json" # modify to git path when new data is pushed 
tally = read_json_file(tally_file_path)

# get all the contest names stored in the first layer
contest_names = list(tally.keys())

# loop over each contest 
for contest_name in contest_names: 
    error = False
    print(contest_name)
    
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        print("     " + selection_name)
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_A = int(selection['message']['pad'])
        big_B = int(selection['message']['data'])
        
        # loop over each guardian's share of decryption
        for i in range(len(shares)):
            share = shares[i]
            M_i = int(share['share'])
            proof = share['proof']
            v_i = int(proof['response'])
            a_i = int(proof['pad'])
            b_i = int(proof['data'])
            c_i = int(proof['challenge'])
            k_i = int(get_field_ij(i, 0, 'k'))
            
            # calculate using given values
            c_i_expected = hash_elems(QBar, big_A, big_B, a_i, b_i, M_i)
            #print("actual c   " + str(c_i))
            #print("expected c " + str(c_i_expected))
            equ_1_left = pow(g, v_i, p)
            equ_1_right = a_i % p * pow(k_i, c_i, p) % p
            equ_2_left = pow(big_A, v_i, p)
            equ_2_right = b_i % p * pow(M_i, c_i, p) % p
            #print("equation1 left   " + str(equ_1_left))
            #print("equation1 right   " + str(equ_1_right))
            
            # confirm that v_i is in the set of Z_q
            if not is_valid_of_Zq(v_i):
                error = True
                print("       v_i error " + str(error))
                
            # confirm that a_i and b_i are both in the set Z_r_q
            if not is_valid_of_Zr_boundary(a_i, q, big_A):
                error = True
                print("       a_i error " + str(error))
            if not is_valid_of_Zr_boundary(b_i, q, big_A):
                error = True
                print("       b_i error " + str(error))
                
            # confirm challege value c_i = H(QBar, (A,B), (a_i, b_i), M_i)
            if not equals(c_i, c_i_expected):
                error = True
                print("       c_i error " + str(error))
                
            # check first equation, g^v_i = a_i * K_i^c_i
            if not equals(equ_1_left, equ_1_right):
                error = True
                print("       equation1 error " + str(error))
                
            # check second equation, A^v_i = b_i * M_i^c_i
            if not equals(equ_2_left, equ_2_right):
                error = True
                print("       equation2 error " + str(error))
                
            
        
if not error:
    print("success")

president-vice-president-contest
     barchi-hallaren-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     cramer-vuocolo-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error

       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     sharp-althea-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     greher-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i

       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
congress-district-5-contest
     soliz-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     keller-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 

       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     parks-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     savannah-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i erro

       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
     exeter-utility-district-referendum-selection
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True
       a_i error True
       b_i error True
       c_i error True
       equation1 error True


In [21]:
# TODO: verify big_A, big_B by getting products from 100 ballot files

In [19]:
# green box 9 
error = False

for contest_name in contest_names: 
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    #print(contest_name) 
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        #print("     " + selection_name)
        m_product = 1
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_B = int(selection['message']['data'])
        big_M = int(selection['value'])
        t = int(selection['tally']) 
        
        for share in shares: 
            M_i = int(share['share'])
            m_product *= M_i % p
            
        # check equation B = M * (M_i products) mod p
        if not equals(big_B % p, big_M * m_product % p):
            error = True
            print(error)
            
        # check equation M = g^t mod p
        if not equals(big_M, pow(g, t, p)):
            error = True
            print(error)
if not error:
    print("success")

success
